In [25]:
import os.path as path
from pathlib import Path

import numpy as np
import pandas as pd

import anndata as ad
import scanpy as sc
# import squidpy as sq

import csv
import gzip
import os
import scipy.io

import cv2

from PIL import Image

#Part 1: 

Here, we read the Visium feature-barcode matrix, which tells us the level of gene expression at each spatial point on the Visium image. (I think)

In [26]:
cd = os.getcwd()
matrix_dir_path = os.path.join(cd, "filtered_feature_bc_matrix")

mat_filtered = scipy.io.mmread(path.join(matrix_dir_path, "matrix.mtx.gz"))

# list of transcript ids, e.g. 'ENSG00000187634'
features_path = path.join(matrix_dir_path, "features.tsv.gz")
feature_ids = [row[0]  for row  in csv.reader(gzip.open(features_path, mode="rt"), delimiter="\t")]

# list of gene names, e.g. 'SAMD11'
gene_names = [row[1]  for row  in csv.reader(gzip.open(features_path, mode="rt"), delimiter="\t")]

# list of feature_types, e.g. 'Gene Expression'
feature_types = [row[2]  for row  in csv.reader(gzip.open(features_path, mode="rt"), delimiter="\t")]

# list of barcodes, e.g. 'AAACATACAAAACG-1'
barcodes_path = os.path.join(matrix_dir_path, "barcodes.tsv.gz")
barcodes = [row[0]  for row  in csv.reader(gzip.open(barcodes_path, mode="rt"), delimiter="\t")]


Now, we load the decompressed data into a single combined feature-barcode matrix for easier processing.

In [27]:
fbc_matrix = pd.DataFrame.sparse.from_spmatrix(mat_filtered)
fbc_matrix.columns = barcodes
fbc_matrix.insert(loc=0, column="feature_id", value=feature_ids)
fbc_matrix.insert(loc=1, column="gene", value=gene_names)
fbc_matrix.insert(loc=2, column="feature_type", value=feature_types)

In [28]:
fbc_matrix

,feature_id,gene,feature_type,AACAATGGAACCACAT-1,AACAATGTGCTCCGAG-1,AACACCAGCCTACTCG-1,AACACCATTCGCATAC-1,AACACCGAATGTCTCA-1,AACACGCAGATAACAA-1,AACACTCGTGAGCTTC-1,...,TGTTCGCTTCTAATCC-1,TGTTCGTACACGGCCA-1,TGTTCGTGGCGTCGTG-1,TGTTGCCAGTCGCCTG-1,TGTTGCCGTTCGACCA-1,TGTTGGCCTGTAGCGG-1,TGTTGGTGCGCACGAG-1,TGTTGGTGCGCTTCGC-1,TGTTGGTGCGGAATCA-1,TGTTGGTGGACTCAGG-1
0,ENSG00000187634,SAMD11,Gene Expression,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ENSG00000188976,NOC2L,Gene Expression,1,0,0,0,0,0,1,...,0,0,0,0,0,1,1,1,0,0
2,ENSG00000187961,KLHL17,Gene Expression,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ENSG00000187583,PLEKHN1,Gene Expression,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ENSG00000187642,PERM1,Gene Expression,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18115,CD4,CD4,Antibody Capture,68679,33584,39114,54248,48609,56826,61884,...,43931,62451,49115,53829,41438,70051,74627,66590,61814,51561
18116,ITGAM,ITGAM,Antibody Capture,6525,37556,31822,6282,6487,11115,26053,...,12896,5484,4798,5626,11187,6949,7982,6373,7497,10556
18117,CD27,CD27,Antibody Capture,22402,15723,19114,25501,23853,29594,23057,...,21310,28071,26988,25196,22841,25997,27928,29879,28185,17410
18118,CCR7,CCR7,Antibody Capture,34870,20814,22500,30801,29853,32579,32568,...,22137,31986,21805,25120,24928,34779,35177,33277,29445,24156


In [29]:
fbc_matrix.describe

<bound method NDFrame.describe of             feature_id     gene      feature_type  AACAATGGAACCACAT-1  \
0      ENSG00000187634   SAMD11   Gene Expression                   0   
1      ENSG00000188976    NOC2L   Gene Expression                   1   
2      ENSG00000187961   KLHL17   Gene Expression                   0   
3      ENSG00000187583  PLEKHN1   Gene Expression                   0   
4      ENSG00000187642    PERM1   Gene Expression                   0   
...                ...      ...               ...                 ...   
18115              CD4      CD4  Antibody Capture               68679   
18116            ITGAM    ITGAM  Antibody Capture                6525   
18117             CD27     CD27  Antibody Capture               22402   
18118             CCR7     CCR7  Antibody Capture               34870   
18119            CD274    CD274  Antibody Capture               28441   

       AACAATGTGCTCCGAG-1  AACACCAGCCTACTCG-1  AACACCATTCGCATAC-1  \
0                   

Now that we have the transcriptomic data, we will load in the corresponding full-res Xenium image and attempt to perform cell segmentation on it.

In [30]:
tissue_positions_df = pd.read_csv('spatial/tissue_positions.csv')

tissue_positions_df.head()

tissue_positions_df_updated = tissue_positions_df.drop(['pxl_row_in_fullres', 'pxl_col_in_fullres'], axis=1)

tissue_positions_df_updated['feature'] = "NAN"

tissue_positions_df_updated.head()

tissue_positions_df_updated.describe()

,in_tissue,array_row,array_col
count,14336.000000,14336.000000,14336.000000
mean,0.401507,63.500000,111.500000
std,0.490220,36.950578,64.664841
min,0.000000,0.000000,0.000000
25%,0.000000,31.750000,55.750000
50%,0.000000,63.500000,111.500000
75%,1.000000,95.250000,167.250000
max,1.000000,127.000000,223.000000


# Load Image and Break it 

Segement image into rows and columns to analyse the contrast and features of each image sector.

In [31]:
image_path = 'spatial/detected_tissue_image.jpg'
img = Image.open(image_path)

img_array = np.array(img)

n_rows = tissue_positions_df.array_row.max() + 1 
n_cols = tissue_positions_df.array_col.max() + 1

cell_height = img_array.shape[0] // n_rows
minicell_width = img_array.shape[0] // (n_cols * 2)

features = np.zeros((n_rows, n_cols))

for row in range(n_rows):    
    for col in range(n_cols):
        if row % 2 == 1: # odd row
            if col % 2 == 1: # even col == cell
                start_col = col * 2 * minicell_width
                end_col = col * 2 * minicell_width + 2 * minicell_width
            else: # odd col == space
                start_col = col * 2 * minicell_width + minicell_width
                end_col = col * 2 * minicell_width + 2 * minicell_width
        else: # even row
            if col % 2 == 0: # odd col == cell
                start_col = col * 2 * minicell_width
                end_col = col * 2 * minicell_width + 2 * minicell_width
            else: # even col == space
                start_col = col * 2 * minicell_width + minicell_width
                end_col = col * 2 * minicell_width + 2 * minicell_width
            
        start_row = row * cell_height
        end_row = (row + 1) * cell_height
        
        cell = img_array[start_row:end_row, start_col:end_col]
    
        feature = np.mean(cell)
        features[row, col] = feature

print(features)
print(features.shape)


[[ 18.16304348   8.4057971   16.43115942 ...  10.31884058  12.27898551
   21.70289855]
 [ 20.86956522  30.79710145  10.13043478 ...  25.88405797  17.78985507
   13.64855072]
 [ 19.31884058  24.01449275  13.55797101 ...  18.76811594  12.77898551
   25.36956522]
 ...
 [234.7173913  234.62318841 235.24637681 ... 228.76086957 227.91304348
  227.26449275]
 [160.5326087  163.93478261 163.94202899 ... 225.7173913  224.63405797
  200.16666667]
 [ 17.37681159  15.18115942  16.15217391 ...  63.96014493  35.73913043
   21.80434783]]
(128, 224)


# Merge datasets and select needed aspects

- Merged the barcode to feature

In [32]:
for index, row in tissue_positions_df_updated.iterrows():
    rowvalue = tissue_positions_df_updated.at[index, 'array_row']
    colvalue = tissue_positions_df_updated.at[index, 'array_col']
    tissue_positions_df_updated.at[index, 'feature'] = features[rowvalue][colvalue]

tissue_positions_df_updated = tissue_positions_df_updated.drop(['array_row', 'array_col'], axis=1)

tissue_positions_df_updated = tissue_positions_df_updated[tissue_positions_df_updated['in_tissue'] != 0]

tissue_positions_df_updated

,barcode,in_tissue,feature
1766,CTAGTTGTGCTGGCGT-1,1,209.731884
1767,GTCACTAGGCATGGTG-1,1,211.278986
1768,GTACTTAGACCTCCTG-1,1,211.047101
1769,CCACCGACTATCGCAT-1,1,212.699275
1877,AAGATAACTTGGAGCC-1,1,205.644928
...,...,...,...
13705,CTACAGTGCGTTAGTG-1,1,237.134058
13813,CGAGCGTTAAGAGGAT-1,1,237.82971
13814,ATTGATGTGCAAGCGA-1,1,238.181159
13815,TGTCAGATAATCTAGG-1,1,237.797101


In [34]:
fbc_matrix = fbc_matrix.drop(['gene', 'feature_type'], axis=1)
transposed_fbc_matrix = fbc_matrix.transpose()

transposed_fbc_matrix

,0,1,2,3,4,5,6,7,8,9,...,18110,18111,18112,18113,18114,18115,18116,18117,18118,18119
feature_id,ENSG00000187634,ENSG00000188976,ENSG00000187961,ENSG00000187583,ENSG00000187642,ENSG00000188290,ENSG00000187608,ENSG00000188157,ENSG00000237330,ENSG00000131591,...,MS4A1,CD3E,CD14,CD40,PECAM1,CD4,ITGAM,CD27,CCR7,CD274
AACAATGGAACCACAT-1,0,1,0,0,0,0,2,4,0,0,...,1439,11645,20584,18084,7759,68679,6525,22402,34870,28441
AACAATGTGCTCCGAG-1,0,0,0,0,0,0,0,1,0,0,...,4480,7169,25967,23564,10061,33584,37556,15723,20814,12199
AACACCAGCCTACTCG-1,0,0,0,0,0,0,1,0,0,0,...,2656,7968,30104,28125,15781,39114,31822,19114,22500,19479
AACACCATTCGCATAC-1,0,0,0,0,0,0,0,4,0,0,...,1653,10671,18977,15611,7545,54248,6282,25501,30801,45090
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TGTTGGCCTGTAGCGG-1,0,1,0,0,0,0,1,3,0,0,...,1750,14339,18966,24810,6857,70051,6949,25997,34779,27523
TGTTGGTGCGCACGAG-1,0,1,0,0,0,1,42,2,0,1,...,1542,13667,28360,19590,7141,74627,7982,27928,35177,25738
TGTTGGTGCGCTTCGC-1,0,1,0,0,0,0,4,3,0,1,...,1783,11000,21048,18156,7409,66590,6373,29879,33277,31783
TGTTGGTGCGGAATCA-1,0,0,0,0,0,2,3,4,0,0,...,1617,10339,15817,17042,5889,61814,7497,28185,29445,26613


In [38]:
merged_df = pd.merge(tissue_positions_df_updated, transposed_fbc_matrix, left_on="barcode", right_on="feature_id")

KeyError: 'feature_id'